# Groundhog Agent - Function Demonstration & Tests

This notebook demonstrates the `groundhog_agent` dispatcher and its supporting helper functions.
These functions wrap the [groundhog](https://groundhog.readthedocs.io/) geotechnical library
and are designed to be called by an LLM in Palantir Foundry AIP Agent Studio.

**Three functions are exposed:**
1. `groundhog_list_methods()` - Browse available calculation methods
2. `groundhog_describe_method()` - Get detailed docs for a specific method
3. `groundhog_agent()` - Run a calculation

**50 methods across 4 categories:**
- Phase Relations (14)
- SPT Correlations (10)
- CPT Correlations (16)
- Bearing Capacity (10)

In [ ]:
from groundhog_agent import groundhog_agent, groundhog_list_methods, groundhog_describe_method
import json

def pp(result):
    """Pretty-print a result dict."""
    print(json.dumps(result, indent=2, default=str))

---
## 1. Discovering Available Methods

The LLM calls `groundhog_list_methods()` first to see what calculations are available.

In [ ]:
# List ALL available methods
all_methods = groundhog_list_methods()
for category, methods in all_methods.items():
    print(f"\n=== {category} ({len(methods)} methods) ===")
    for name, brief in methods.items():
        print(f"  {name}: {brief}")

In [ ]:
# Filter to a specific category
spt_methods = groundhog_list_methods("SPT Correlations")
pp(spt_methods)

---
## 2. Getting Detailed Method Documentation

Before running a calculation, the LLM calls `groundhog_describe_method()` to understand
the exact parameters, their types, ranges, defaults, and what the method returns.

In [ ]:
# Detailed docs for bulk_unit_weight
pp(groundhog_describe_method("bulk_unit_weight"))

In [ ]:
# Detailed docs for a CPT method
pp(groundhog_describe_method("cpt_normalisations"))

---
## 3. Phase Relations - Worked Examples

Basic soil property conversions: void ratio, porosity, unit weight, water content, etc.

In [ ]:
# Porosity of 0.4 -> Void ratio
result = groundhog_agent("voidratio_from_porosity", {"porosity": 0.4})
print("Porosity 0.4 -> Void ratio:", result)
# e = 0.4 / (1 - 0.4) = 0.667

In [ ]:
# Bulk unit weight of a fully saturated soil with e=0.65, Gs=2.65
result = groundhog_agent("bulk_unit_weight", {
    "saturation": 1.0,
    "voidratio": 0.65,
    "specific_gravity": 2.65
})
print("Saturated soil (e=0.65, Gs=2.65):")
print(f"  Bulk unit weight:      {result['bulk unit weight [kN/m3]']:.2f} kN/m3")
print(f"  Effective unit weight:  {result['effective unit weight [kN/m3]']:.2f} kN/m3")

In [ ]:
# Relative density: e=0.55, e_min=0.4, e_max=0.9
result = groundhog_agent("relative_density", {
    "void_ratio": 0.55,
    "e_min": 0.4,
    "e_max": 0.9
})
print(f"Relative density: {result['Dr [-]']:.2f} ({result['Dr [-]']*100:.0f}%)")

In [ ]:
# Back-calculate void ratio and water content from bulk unit weight
result = groundhog_agent("voidratio_from_bulk_unit_weight", {
    "bulkunitweight": 19.0,
    "saturation": 1.0,
    "specific_gravity": 2.65
})
print(f"Bulk unit weight 19.0 kN/m3 (saturated):")
print(f"  Void ratio:     {result['e [-]']:.3f}")
print(f"  Water content:  {result['w [-]']:.3f} ({result['w [-]']*100:.1f}%)")

In [ ]:
# Unit conversion: density <-> unit weight
r1 = groundhog_agent("density_from_unit_weight", {"gamma": 20.0})
r2 = groundhog_agent("unit_weight_from_density", {"density": 2000.0})
print(f"20.0 kN/m3 = {r1['Density [kg/m3]']:.1f} kg/m3")
print(f"2000 kg/m3 = {r2['Unit weight [kN/m3]']:.2f} kN/m3")

---
## 4. SPT Correlations - Worked Examples

Standard Penetration Test data interpretation.

In [ ]:
# Step 1: Correct raw SPT N to N60
# US rig, safety hammer, rope & pulley, 100mm borehole, 12m rod length
n60_result = groundhog_agent("spt_N60_correction", {
    "N": 25,
    "borehole_diameter": 100.0,
    "rod_length": 12.0,
    "country": "United States",
    "hammertype": "Safety",
    "hammerrelease": "Rope and pulley"
})
print("SPT N60 Correction:")
print(f"  Raw N = 25 -> N60 = {n60_result['N60 [-]']:.1f}")
print(f"  Hammer efficiency: {n60_result['eta_H [%]']}%")
print(f"  Rod length factor: {n60_result['eta_R [-]']}")

In [ ]:
# Step 2: Apply overburden correction to get (N1)60
N60 = n60_result['N60 [-]']
overburden_result = groundhog_agent("spt_overburden_correction_liaowhitman", {
    "N": N60,
    "sigma_vo_eff": 120.0  # kPa
})
print(f"Overburden correction at sigma'vo = 120 kPa:")
print(f"  CN = {overburden_result['CN [-]']:.3f}")
print(f"  (N1)60 = {overburden_result['N1 [-]']:.1f}")

In [ ]:
# Step 3: Estimate friction angle from SPT
phi_result = groundhog_agent("spt_friction_angle_kulhawymayne", {
    "N": 25.0,
    "sigma_vo_eff": 120.0
})
print(f"Friction angle (Kulhawy & Mayne): {phi_result['Phi [deg]']:.1f} deg")

# Compare with PHT method using (N1)60
N1_60 = overburden_result['N1 [-]']
phi_pht = groundhog_agent("spt_friction_angle_pht", {"N1_60": N1_60})
print(f"Friction angle (PHT):             {phi_pht['Phi [deg]']:.1f} deg")

In [ ]:
# Classify soil from raw SPT N
print("Cohesionless classification:")
for N in [3, 8, 20, 35, 55]:
    r = groundhog_agent("spt_relative_density_class", {"N": float(N)})
    print(f"  N = {N:2d} -> {r['Dr class']}")

print("\nCohesive soil classification:")
for N in [1, 3, 6, 12, 22, 35]:
    r = groundhog_agent("spt_consistency_class", {"N": float(N)})
    print(f"  N = {N:2d} -> {r['Consistency class']:12s}  (qu: {r['qu min [kPa]']}-{r['qu max [kPa]']} kPa)")

In [ ]:
# Young's modulus from SPT by soil type
N1_60_val = 30.0
for soil in ["Silts", "Clean sands", "Coarse sands", "Gravels"]:
    r = groundhog_agent("spt_youngs_modulus_aashto", {"N1_60": N1_60_val, "soiltype": soil})
    print(f"  {soil:15s}: Es = {r['Es [MPa]']:.1f} MPa")

In [ ]:
# Undrained shear strength from SPT (cohesive soil)
r = groundhog_agent("spt_undrained_shear_strength_salgado", {
    "pi": 35.0,   # Plasticity index = 35%
    "N_60": 12.0
})
print(f"Su from SPT (PI=35%, N60=12): {r['Su [kPa]']:.1f} kPa")
print(f"  alpha' factor: {r['alpha_prime [-]']:.4f}")

---
## 5. CPT Correlations - Worked Examples

Cone Penetration Test data interpretation: normalization, soil classification, strength and stiffness estimation.

In [ ]:
# Step 1: Normalize raw CPT data
# Raw readings: qc=5 MPa, fs=0.05 MPa, u2=0.2 MPa at 5m depth
# Total stress=100 kPa, effective stress=60 kPa, cone area ratio=0.8
cpt_raw = groundhog_agent("cpt_normalisations", {
    "measured_qc": 5.0,
    "measured_fs": 0.05,
    "measured_u2": 0.2,
    "sigma_vo_tot": 100.0,
    "sigma_vo_eff": 60.0,
    "depth": 5.0,
    "cone_area_ratio": 0.8,
})
print("CPT Normalisation Results:")
print(f"  qt (corrected cone resistance): {cpt_raw['qt [MPa]']:.3f} MPa")
print(f"  Rf (friction ratio):            {cpt_raw['Rf [pct]']:.2f}%")
print(f"  Qt (normalised cone resistance): {cpt_raw['Qt [-]']:.1f}")
print(f"  Bq (pore pressure ratio):       {cpt_raw['Bq [-]']:.4f}")
print(f"  Ic (soil behaviour index):       {cpt_raw['Ic [-]']:.2f}")
print(f"  Classification:                  {cpt_raw['Ic class']}")

In [ ]:
# Step 2: Soil classification from Ic
for ic_val in [1.2, 1.8, 2.3, 2.8, 3.2, 3.8]:
    r = groundhog_agent("cpt_soil_class_robertson", {"ic": ic_val})
    print(f"  Ic = {ic_val:.1f} -> Zone {r['Soil type number [-]']}: {r['Soil type']}")

In [ ]:
# Friction angle and relative density for sand from CPT
phi_cpt = groundhog_agent("cpt_friction_angle_sand", {
    "qt": 10.0,
    "sigma_vo_eff": 100.0,
})
print(f"CPT friction angle (qt=10 MPa, sigma'vo=100 kPa): {phi_cpt['Phi [deg]']:.1f} deg")

# Relative density for NC sand (Baldi)
dr_nc = groundhog_agent("cpt_relative_density_nc_sand", {
    "qc": 15.0,
    "sigma_vo_eff": 100.0,
})
print(f"Relative density NC sand (qc=15 MPa): {dr_nc['Dr [-]']:.2f} ({dr_nc['Dr [-]']*100:.0f}%)")

# Relative density for OC sand (Baldi) - needs K0
dr_oc = groundhog_agent("cpt_relative_density_oc_sand", {
    "qc": 15.0,
    "sigma_vo_eff": 100.0,
    "k0": 1.2,
})
print(f"Relative density OC sand (K0=1.2):    {dr_oc['Dr [-]']:.2f} ({dr_oc['Dr [-]']*100:.0f}%)")

In [ ]:
# Undrained shear strength from CPT (clay)
su_cpt = groundhog_agent("cpt_undrained_shear_strength", {
    "qnet": 0.5,   # Net cone resistance in MPa
    "Nk": 15.0,    # Cone factor (calibrate to lab tests)
})
print(f"Su from CPT (qnet=0.5 MPa, Nk=15): {su_cpt['Su [kPa]']:.1f} kPa")

# OCR from CPT
ocr_cpt = groundhog_agent("cpt_ocr", {"Qt": 5.0, "Bq": 0.5})
print(f"\nOCR from CPT (Qt=5, Bq=0.5):")
print(f"  Qt-based: {ocr_cpt['OCR_Qt_BE [-]']:.2f} (range {ocr_cpt['OCR_Qt_LE [-]']:.2f} - {ocr_cpt['OCR_Qt_HE [-]']:.2f})")
print(f"  Bq-based: {ocr_cpt['OCR_Bq_BE [-]']:.2f} (range {ocr_cpt['OCR_Bq_LE [-]']:.2f} - {ocr_cpt['OCR_Bq_HE [-]']:.2f})")

# Small-strain shear modulus
gmax_sand = groundhog_agent("cpt_gmax_sand", {"qc": 10.0, "sigma_vo_eff": 100.0})
gmax_clay = groundhog_agent("cpt_gmax_clay", {"qc": 2.0})
print(f"\nGmax (sand, qc=10 MPa): {gmax_sand['Gmax [kPa]']:.0f} kPa ({gmax_sand['Gmax [kPa]']/1000:.1f} MPa)")
print(f"Gmax (clay, qc=2 MPa):  {gmax_clay['Gmax [kPa]']:.0f} kPa ({gmax_clay['Gmax [kPa]']/1000:.1f} MPa)")

In [ ]:
# Unit weight and constrained modulus from CPT
gamma_cpt = groundhog_agent("cpt_unit_weight", {"ft": 0.05, "sigma_vo_eff": 100.0})
print(f"Unit weight from CPT (ft=0.05 MPa): {gamma_cpt['gamma [kN/m3]']:.2f} kN/m3")

M_cpt = groundhog_agent("cpt_constrained_modulus", {
    "qt": 10.0, "ic": 2.0, "sigma_vo": 150.0, "sigma_vo_eff": 100.0,
})
print(f"Constrained modulus (qt=10, Ic=2.0): {M_cpt['M [kPa]']:.0f} kPa ({M_cpt['M [kPa]']/1000:.1f} MPa)")

# K0 for sand from CPT
k0_cpt = groundhog_agent("cpt_k0_sand", {"qt": 10.0, "sigma_vo_eff": 100.0, "ocr": 1.0})
print(f"K0 from CPT (qt=10, OCR=1.0): {k0_cpt['K0 CPT [-]']:.3f}")
print(f"K0 conventional:              {k0_cpt['K0 conventional [-]']:.3f}")

---
## 6. Bearing Capacity - Worked Examples

Shallow foundation bearing capacity calculations per API RP 2GEO.

In [ ]:
# Bearing capacity factors vs friction angle
print("Bearing Capacity Factors:")
print(f"  {'phi':>4s}  {'Nq':>8s}  {'Ng(Vesic)':>10s}  {'Ng(Meyerhof)':>13s}  {'Ng(D&B rough)':>14s}")
print("  " + "-" * 56)
for phi in [25, 30, 35, 40, 45]:
    nq = groundhog_agent("bearing_capacity_nq", {"friction_angle": float(phi)})
    ng_v = groundhog_agent("bearing_capacity_ngamma_vesic", {"friction_angle": float(phi)})
    ng_m = groundhog_agent("bearing_capacity_ngamma_meyerhof", {"friction_angle": float(phi)})
    ng_db = groundhog_agent("bearing_capacity_ngamma_davisbooker", {
        "friction_angle": float(phi), "roughness_factor": 1.0,
    })
    print(f"  {phi:4d}  {nq['Nq [-]']:8.1f}  {ng_v['Ngamma [-]']:10.1f}  {ng_m['Ngamma [-]']:13.1f}  {ng_db['Ngamma [-]']:14.1f}")

In [ ]:
# Undrained bearing capacity: 10m x 10m skirted foundation on clay
bc_undrained = groundhog_agent("bearing_capacity_undrained_api", {
    "effective_length": 10.0,
    "effective_width": 10.0,
    "su_base": 50.0,
    "base_depth": 2.0,
    "skirted": True,
})
print("Undrained Bearing Capacity (10m x 10m, Su=50 kPa, D=2m):")
print(f"  qu = {bc_undrained['qu [kPa]']:.1f} kPa")
print(f"  Vertical capacity = {bc_undrained['vertical_capacity [kN]']:.0f} kN")
print(f"  Kc = {bc_undrained['K_c [-]']:.3f} (shape: {bc_undrained['s_c [-]']:.3f}, depth: {bc_undrained['d_c [-]']:.3f})")

In [ ]:
# Drained bearing capacity: 10m x 10m foundation on sand
bc_drained = groundhog_agent("bearing_capacity_drained_api", {
    "vertical_effective_stress": 100.0,
    "effective_friction_angle": 35.0,
    "effective_unit_weight": 9.0,
    "effective_length": 10.0,
    "effective_width": 10.0,
    "base_depth": 2.0,
})
print("Drained Bearing Capacity (10m x 10m, phi'=35, D=2m):")
print(f"  qu = {bc_drained['qu [kPa]']:.1f} kPa")
print(f"  Vertical capacity = {bc_drained['vertical_capacity [kN]']:.0f} kN")
print(f"  Nq = {bc_drained['N_q [-]']:.1f}, Ngamma = {bc_drained['N_gamma [-]']:.1f}")

In [ ]:
# Effective area with eccentric loading
eff_rect = groundhog_agent("effective_area_rectangle", {
    "length": 10.0,
    "width": 5.0,
    "eccentricity_length": 1.0,
    "eccentricity_width": 0.5,
})
print("Effective Area (Rectangular, L=10m, B=5m, e1=1.0m, e2=0.5m):")
print(f"  L' = {eff_rect['effective_length [m]']:.1f} m, B' = {eff_rect['effective_width [m]']:.1f} m")
print(f"  A' = {eff_rect['effective_area [m2]']:.1f} m2 (original: 50 m2)")

eff_circ = groundhog_agent("effective_area_circle", {
    "foundation_radius": 5.0,
    "eccentricity": 1.0,
})
print(f"\nEffective Area (Circular, R=5m, e=1.0m):")
print(f"  L' = {eff_circ['effective_length [m]']:.2f} m, B' = {eff_circ['effective_width [m]']:.2f} m")
print(f"  A' = {eff_circ['effective_area [m2]']:.1f} m2 (original: {3.14159*25:.1f} m2)")

---
## 7. Error Handling

The dispatcher gracefully handles invalid method names and out-of-range inputs.

In [ ]:
# Unknown method -> returns error dict
r = groundhog_agent("nonexistent_method", {"x": 1})
print("Unknown method:", r["error"][:80], "...")

# Out-of-range input (porosity > 1.0) -> groundhog returns None
r = groundhog_agent("voidratio_from_porosity", {"porosity": 5.0})
print("\nOut-of-range (porosity=5.0):", r)

# Wrong parameter names -> groundhog returns None (uses defaults for missing params)
r = groundhog_agent("bulk_unit_weight", {"S": 1.0, "e": 0.65})
print("\nWrong param names (returns None):", r)

---
## 8. Full LLM Workflow Simulation

This simulates what the LLM would do in Foundry when a user asks:

> *"I have an SPT N value of 18 at 8m depth in a medium sand. The unit weight is 19 kN/m3
> and the water table is at 3m. What's the friction angle and relative density?"*

In [ ]:
# LLM Step 1: List methods to find relevant SPT correlations
methods = groundhog_list_methods("SPT Correlations")
print("Available SPT methods:")
for name, brief in methods["SPT Correlations"].items():
    print(f"  - {name}: {brief}")

In [ ]:
# LLM Step 2: Get details on the methods it wants to use
desc = groundhog_describe_method("spt_N60_correction")
print(f"Method: {desc['brief']}")
print(f"Required params: {[k for k,v in desc['parameters'].items() if v.get('required')]}")

In [ ]:
# LLM Step 3: Calculate
# Given: N=18, depth=8m, gamma=19 kN/m3, water table at 3m
depth = 8.0  # m
gamma = 19.0  # kN/m3
wt_depth = 3.0  # m
gamma_w = 10.0  # kN/m3

# Effective overburden stress
sigma_vo_eff = gamma * wt_depth + (gamma - gamma_w) * (depth - wt_depth)
print(f"Effective overburden stress: {sigma_vo_eff:.1f} kPa")

# Correct N to N60 (assuming US Safety hammer, standard setup)
n60 = groundhog_agent("spt_N60_correction", {
    "N": 18.0,
    "borehole_diameter": 100.0,
    "rod_length": depth,
    "country": "United States",
    "hammertype": "Safety",
    "hammerrelease": "Rope and pulley"
})
print(f"N60 = {n60['N60 [-]']:.1f}")

# Overburden correction
n1 = groundhog_agent("spt_overburden_correction_liaowhitman", {
    "N": n60['N60 [-]'],
    "sigma_vo_eff": sigma_vo_eff
})
print(f"(N1)60 = {n1['N1 [-]']:.1f}")

# Friction angle
phi = groundhog_agent("spt_friction_angle_kulhawymayne", {
    "N": 18.0,
    "sigma_vo_eff": sigma_vo_eff
})
print(f"Friction angle: {phi['Phi [deg]']:.1f} deg")

# Relative density (assuming D50 = 0.5mm for medium sand)
dr = groundhog_agent("spt_relative_density_kulhawymayne", {
    "N1_60": n1['N1 [-]'],
    "d_50": 0.5
})
print(f"Relative density: {dr['Dr [pct]']:.0f}%")

# Classification
cls = groundhog_agent("spt_relative_density_class", {"N": 18.0})
print(f"Classification: {cls['Dr class']}")